In [1]:

%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import get_state as getstate
import utility as util
import plotfig as fig
import ground_state as gs
import lanczos
import time
from tqdm.notebook import tqdm
start_time = time.time()
M_PI = math.pi
                  
#####################################
def compute_Aw_main(A,epx,epy,tpd,tpddiff,tpp,pds,pdsdiff,pdp,pdpdiff,pps,ppp,Upp,Udif,Vpp,Upd,Updf,\
                    d_double,pUppx_double,pUppy_double,pVpp_double,pUpdx_double,pUpdy_double,U, S_val, Sz_val, AorB_sym): 
    diff = round(abs(epx-epy)*0.5,3)
    if Norb==7:
        fname = 'epx'+str(epx)+'epy'+str(epy)+'_tpd'+str(tpd)+'_tpddiff'+str(tpddiff)+'_tpp'+str(tpp) \
                  +'_A'+str(A)+'_B'+str(pam.B)+'_C'+str(pam.C)+'_Upp'+str(Upp)+'_Udif'+str(Udif)+'_Vpp'\
                  +str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_tpd'+str(tpd)+'_tpddiff'+str(tpddiff)+'_tpp'+str(tpp)+'_diff'\
                  +str(diff)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                  +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)+'_Mc'+str(Mc)+'_eta'+str(eta)
    elif Norb==9 or Norb==10 or Norb==11:
        fname = 'epx'+str(epx)+'epy'+str(epy)+'_pds'+str(pds)+'_pdsdiff'+str(pdsdiff)+'_pdp'+str(pdp)\
                  +'_pdpdiff'+str(pdpdiff)+'_pps'+str(pps)+'_ppp'+str(ppp) \
                  +'_A'+str(B)+'_B'+str(pam.B)+'_C'+str(pam.C)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                  +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)\
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_pds'+str(pds)+'_pdsdiff'+str(pdsdiff)+'_pdp'+str(pdp)+'_pdpdiff'+str(pdpdiff)\
                  +'_pps'+str(pps)+'_ppp'+str(ppp)+'_diff'+str(diff)+'_Upp'+str(Upp)+'_Udif'+str(Udif)\
                  +'_Vpp'+str(Vpp)+'_Upd'+str(Upd)+'_Updf'+str(Updf)+'_Mc'+str(Mc)+'_eta'+str(eta)
                
    w_vals = np.arange(pam.wmin, pam.wmax, pam.eta/4.0)
   
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))

    # set up H0
    if Norb==7:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,tpd,tpddiff,tpp,0,0,0,0)
    elif Norb==9 or Norb==11:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,0,0,0,pds,pdsdiff,pdp,pdpdiff,pps,ppp)
            
    T_pd   = ham.create_tpd_nn_matrix(VS,tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac)
    T_pp   = ham.create_tpp_nn_matrix(VS,tpp_nn_hop_fac)  
    Esite  = ham.create_edep_diag_matrix(VS,A,epx,epy)
    
    H0 = T_pd + T_pp + Esite  
    
    '''
    Below probably not necessary to do the rotation by multiplying U and U_d
    the basis_change.py is only for label the state as singlet or triplet
    and assign the interaction matrix
    '''
    if pam.if_H0_rotate_byU==1:
        H0_new = U_d.dot(H0.dot(U))
    
    clf()

    if Norb==7 or Norb==9 or Norb==10 or Norb==11:     
        Hint = ham.create_interaction_matrix_ALL_syms(VS,d_double,pUppx_double,pUppy_double,pVpp_double,pUpdx_double,\
                                                      pUpdy_double,S_val, Sz_val,AorB_sym,A,Upp,Udif,Vpp,Upd,Updf)
        if pam.if_H0_rotate_byU==1:
            H = H0_new + Hint 
        else:
            H = H0 + Hint 
        H.tocsr()

        ####################################################################################
        # compute GS only for turning on full interactions
        if pam.if_get_ground_state==1:
            vals, vecs, wgt_d8, wgt_d9L, wgt_d10L2 = gs.get_ground_state(H, VS, S_val, Sz_val)
            if Norb==7:
#                 util.write_GS('Egs_'+flowpeak+'.txt',A,epx,epy,tpd,vals[0])
                util.write_GS_components('GS_weights_'+flowpeak+'.txt',A,epx,epy,tpd,wgt_d8, wgt_d9L, wgt_d10L2)
            elif Norb==9 or Norb==10 or Norb==11:
#                 util.write_GS2('Egs_'+flowpeak+'.txt',A,epx,epy,pds,pdp,vals[0])
                util.write_GS_components2('GS_weights_'+flowpeak+'.txt',A,epx,epy,pds,pdp,wgt_d8, wgt_d9L, wgt_d10L2)
            
        #########################################################################
        '''
        Compute A(w) for various states
        '''
        # compute d8
#         fig.compute_Aw_d8_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, A, w_vals, "Aw_d8_sym_", fname)
        
        # compute d9L
#         b1L_state_indices, a1L_state_indices, b1L_state_labels, a1L_state_labels \
#                 = getstate.get_d9L_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, b1L_state_indices, b1L_state_labels, "Aw_b1L_", fname)
#         fig.compute_Aw1(H, VS, w_vals, a1L_state_indices, a1L_state_labels, "Aw_a1L_", fname)
        
#         # compute d10L2
#         d10L2_state_indices, d10L2_state_labels = getstate.get_d10L2_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, d10L2_state_indices, d10L2_state_labels, "Aw_d10L2_", fname)
        
##########################################################################
if __name__ == '__main__': 
    Mc  = pam.Mc
    print ('Mc=',Mc)

    Norb = pam.Norb
    eta  = pam.eta
    ed   = pam.ed

    As = pam.As
    B  = pam.B
    C  = pam.C
    
    # set up VS
    VS = vs.VariationalSpace(Mc)
    basis.count_VS(VS)
    
    d_double,pUppx_double,pUppy_double, pVpp_double, pUpdx_double,pUpdy_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    if pam.basis_change_type=='all_states':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix(VS,d_double)
       
        if pam.if_print_VS_after_basis_change==1:
            basis.print_VS_after_basis_change(VS,S_val,Sz_val)
    elif pam.basis_change_type=='d_double':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix_d_double(VS,d_double)

    U_d = (U.conjugate()).transpose()
    # check if U if unitary
    #checkU_unitary(U,U_d)
    
    if Norb==7:
        for tpd in tqdm(pam.tpds):
            for tpddiff in pam.tpddiffs:
                tpddiff = round(tpddiff,3)
                for ep1 in pam.ep_avgs: 
                    for ep2 in pam.ep_difs: 
                        epx = ep1+ep2
                        epy = ep1-ep2
                        for A in pam.As:
                            util.get_atomic_d8_energy(A,B,C)
                            for tpp in pam.tpps:
                                for Upp in pam.Upps:
                                    Upp = round(Upp,3)
                                    for Udif in pam.Udifs:
                                        Udif = round(Udif,3)
                                        for Vpp in pam.Vpps:
                                            Vpp = round(Vpp,3)
                                            for Upd in pam.Upds:
                                                Upd = round(Upd,3)
                                                for Updf in pam.Updfs:
                                                    Updf = round(Updf,3) 
                                                    print ('===================================================')
                                                    print ('A=',A, 'epx=', epx, 'epy=', epy, ' tpd=',tpd,' tpddiff=',tpddiff,' tpp=',tpp, \
                                                           ' Upp=',Upp,'Udif=',Udif,'Vpp=',Vpp,'Upd=',Upd,'Updf=',Updf)
                                                    compute_Aw_main(A,epx,epy,tpd,tpddiff,tpp,0,0,0,0,0,0,Upp,Udif,Vpp,Upd,Updf,\
                                                                    d_double,pUppx_double,pUppy_double,\
                                                                    pVpp_double,pUpdx_double,\
                                                                    pUpdy_double,U, S_val, Sz_val, AorB_sym)

                                
    elif Norb==9 or Norb==10 or Norb==11:
        total_iterations =len(pam.pdps)*len(pam.pdsdiffs)*len(pam.ep_avgs)*len(pam.ep_difs)*len(pam.As)*\
        len(pam.Upps)*len(pam.Udifs)*len(pam.Vpps)*len(pam.Upds)*len(pam.Updfs)
        pps = pam.pps
        ppp = pam.ppp
        with tqdm(total=total_iterations, desc="Total Progress") as pbar:
            for ii in range(0,len(pam.pdps)):
                pds = pam.pdss[ii]
                pds = round(pds,5)
                pdp = pam.pdps[ii]
                for pdsdiff in pam.pdsdiffs:
                    pdpdiff = round(pdsdiff*np.sqrt(3)/4,5)
                    pdsdiff = round(pdsdiff,3)
                    for ep1 in pam.ep_avgs: 
                        for ep2 in pam.ep_difs: 
                            epx = ep1+ep2
                            epy = ep1-ep2
                            for A in pam.As:
                                util.get_atomic_d8_energy(A,B,C)
                                for Upp in pam.Upps:
                                    Upp = round(Upp,3)
                                    for Udif in pam.Udifs:
                                        Udif = round(Udif,3)
                                        for Vpp in pam.Vpps:
                                            Vpp = round(Vpp,3)
                                            for Upd in pam.Upds:
                                                Upd = round(Upd,3)
                                                for Updf in pam.Updfs:
                                                    Updf = round(Updf,3)
                                                    print ('===================================================')
                                                    print ('A=',A, 'epx=', epx, 'epy=', epy,' pds=',pds,'pdsdiff=',pdsdiff,' pdp=',pdp,\
                                                           ' pps=',pps,' ppp=',ppp, ' Upp=',Upp, ' Uppdif=',Udif,' Vpp=',Vpp ,' Upd=',Upd,'Upddiff=',Updf)
                                                    compute_Aw_main(A,epx,epy,0,0,0,pds,pdsdiff,pdp,pdpdiff,pps,ppp,Upp,Udif,Vpp,Upd,Updf,\
                                                                    d_double,pUppx_double,pUppy_double,\
                                                                    pVpp_double,pUpdx_double,\
                                                                    pUpdy_double,\
                                                                    U, S_val, Sz_val, AorB_sym)  
                                                    pbar.update(1)


    print("--- %s seconds ---" % (time.time() - start_time))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Ni_orbs =  ['d3z2r2', 'dx2y2', 'dxy', 'dxz', 'dyz']
O1_orbs =  ['px1', 'py1', 'pz1']
O2_orbs =  ['px2', 'py2', 'pz2']
compute A(w) for symmetries =  ['1A1', '1B1', '3B1', '1A2', '3A2', '1E', '3E']
Mc= 10
VS.dim =  93025
No. of states with count_upup, count_updn, count_dnup, count_dndn: 0 46975 46050 0
len(d_list) 25
len(pUppx_list) 450
len(pUppy_list) 450
len(pVpp_list) 3240
len(pUpdx_list) 60
len(pUpdy_list) 60


Total Progress:   0%|          | 0/132 [00:00<?, ?it/s]

E_1S =  12.16
E_1G =  7.76
E_1D =  6.71
E_3P =  7.05
E_3F =  4.8
A= 6.0 epx= 3.5 epy= 3.5  pds= 1.50111 pdsdiff= 0.0  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Uppdif= 0.0  Vpp= 0.0  Upd= 0.5 Upddiff= -0.2
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-1.35099841 -0.30244491 -0.30242293 -0.27883055 -0.27749504 -0.27749504
 -0.27749504 -0.27749504 -0.26799282 -0.17505206]
eigenvalue =  -1.3509984136246922
Compute the weights in GS (lowest Aw peak)
no e-h state  15120 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.015265236790730226
no e-h state  22335 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.029078667508067174
no e-h state  22495 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.013269383121419112
no e-h state  23438 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.009073110600408682
no e-h state  23468 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.009073110600408702
no e

start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-1.34875579 -0.30225774 -0.30224715 -0.27881608 -0.27747219 -0.27747219
 -0.27747219 -0.27747219 -0.26796498 -0.17467673]
eigenvalue =  -1.348755790840508
Compute the weights in GS (lowest Aw peak)
no e-h state  15120 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.014968704790907495
no e-h state  22335 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.029110144044434152
no e-h state  22495 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.013569837582815039
no e-h state  23438 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.008955633370986961
no e-h state  23468 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.008955633370986938
no e-h state  23676 dx2y2 -1 -2 0 px1 0 0 0 S= 0 Sz= 0 , weight =  0.008310785687987233
no e-h state  23706 dx2y2 -1 0 0 px1 0 0 0 S= 0 Sz= 0 , weight =  0.13979326635261077
no e-h state  23736 dx2y2 -1 2 0 px1 0 0 0 S= 0 Sz= 0 , weight 

no e-h state  23978 dx2y2 0 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.127557094594279
no e-h state  24006 dx2y2 0 0 0 dx2y2 0 0 0 S= 0 Sz= 0 , weight =  0.08154808459413498
no e-h state  24751 px1 -1 0 0 py2 0 1 0 S= 0 Sz= 0 , weight =  0.02913053707432208
no e-h state  24915 py2 0 -1 0 py2 0 1 0 S= 0 Sz= 0 , weight =  0.027741048004817748
no e-h state  24937 dx2y2 0 0 0 py2 0 1 0 S= 0 Sz= 0 , weight =  0.12755709459427914
no e-h state  24959 py2 0 1 0 py2 0 1 0 S= 0 Sz= 0 , weight =  0.0138705240024089
no e-h state  32943 dx2y2 0 0 0 px1 1 -2 0 S= 0 Sz= 0 , weight =  0.008461272636877959
no e-h state  33844 px1 -1 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.029341063855541985
no e-h state  34008 px1 0 -1 0 py2 1 0 0 S= 0 Sz= 0 , weight =  0.02913053707432209
no e-h state  34026 dx2y2 0 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.1363990193136213
no e-h state  34056 px1 0 1 0 py2 1 0 0 S= 0 Sz= 0 , weight =  0.02913053707432212
no e-h state  34216 px1 1 0 0 px1 1 0 0 S= 0 Sz= 0 , weight =  0.01467

...

sumweight= 0.970970641200513
A= 6.0 epx= 7.1 epy= 6.9  pds= 1.50111 pdsdiff= 0.0  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Uppdif= 0.0  Vpp= 0.0  Upd= 0.5 Upddiff= -0.16
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[2.2366096  2.36039374 2.70342012 2.7076856  3.14516979 3.41662485
 3.71071356 3.71138131 3.74831672 3.84386453]
eigenvalue =  2.2366095971989512
Compute the weights in GS (lowest Aw peak)
no e-h state  15120 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.00553781410841718
no e-h state  22335 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.012017524940396826
no e-h state  22495 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.005702600603443806
no e-h state  23438 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.005302206642027005
no e-h state  23468 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.005302206642027009
no e-h state  23676 dx2y2 -1 -2 0 px1 0 0 0 S= 0 Sz

sumweight= 0.9709958976858731
A= 6.0 epx= 7.1 epy= 6.9  pds= 1.50111 pdsdiff= 0.0  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Uppdif= 0.0  Vpp= 0.0  Upd= 0.5 Upddiff= -0.1
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[2.23692118 2.36069053 2.70460112 2.70710906 3.14571283 3.41713478
 3.7094262  3.71388766 3.74921226 3.84393536]
eigenvalue =  2.236921177657983
Compute the weights in GS (lowest Aw peak)
no e-h state  15120 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.005396099003618903
no e-h state  22335 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.012019594755855914
no e-h state  22495 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.005854496393505704
no e-h state  23438 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.0052000497557147165
no e-h state  23468 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.005200049755714725
no e-h state  23676 dx2y2 -1 -2 0 px1 0 0 0 S= 0 S

sumweight= 0.9709605743550342
A= 6.0 epx= 7.1 epy= 6.9  pds= 1.50111 pdsdiff= 0.0  pdp= 0.65  pps= 0.9  ppp= 0.2  Upp= 0.0  Uppdif= 0.0  Vpp= 0.0  Upd= 0.5 Upddiff= -0.04
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[2.23635318 2.36048786 2.70521328 2.70596442 3.14524947 3.41684947
 3.70709643 3.71535412 3.74866502 3.84392691]
eigenvalue =  2.2363531751195804
Compute the weights in GS (lowest Aw peak)
no e-h state  15120 px1 -1 0 0 px1 -1 0 0 S= 0 Sz= 0 , weight =  0.0052550421646725895
no e-h state  22335 px1 -1 0 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.01201623171298514
no e-h state  22495 py2 0 -1 0 py2 0 -1 0 S= 0 Sz= 0 , weight =  0.0060069873886072255
no e-h state  23438 dx2y2 -2 -1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.005094715539890848
no e-h state  23468 dx2y2 -2 1 0 py2 0 0 0 S= 0 Sz= 0 , weight =  0.005094715539890848
no e-h state  23676 dx2y2 -1 -2 0 px1 0 0 0 S= 0

sumweight= 0.9708647688201945
--- 1890.1546001434326 seconds ---


<Figure size 640x480 with 0 Axes>